In [2]:
import torch
import torch.nn as nn
from torchinfo import summary
device="cuda" if torch.cuda.is_available else "cpu"

class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.bn1(self.conv1(inputs)))
        x = self.relu(self.bn2(self.conv2(x)))
        return x

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
        return x, p

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2)
        self.conv = conv_block(2 * out_c, out_c)

    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        return x

class segUnet(nn.Module):
    def __init__(self, num_classes, in_channels=3, depth=5, start_filts=64):
        super().__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.start_filts = start_filts
        self.depth = depth

        """ Encoders """
        self.encoders = nn.ModuleList([encoder_block(in_channels, start_filts)])
        self.encoders.extend([encoder_block(start_filts * (2 ** i), start_filts * (2 ** (i + 1))) for i in range(depth - 1)])

        """ Bottleneck """
        self.bottleneck = conv_block(start_filts * (2 ** (depth - 1)), start_filts * (2 ** depth))

        """ Decoders """
        self.decoders = nn.ModuleList([decoder_block(start_filts * (2 ** i), start_filts * (2 ** (i - 1))) for i in range(depth, 0, -1)])

        """ Classifier """
        self.outputs = nn.Conv2d(start_filts, num_classes, kernel_size=1)

    def forward(self, inputs):
        skips = []
        x = inputs
        for encoder in self.encoders:
            x, p = encoder(x)
            skips.append(x)
            x = p

        x = self.bottleneck(x)

        for i, decoder in enumerate(self.decoders):
            x = decoder(x, skips[-(i+1)])

        outputs = self.outputs(x)
        return outputs

# Check for CUDA availability
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate the model and move it to the appropriate device
model = segUnet(num_classes=19, in_channels=3, depth=3, start_filts=8).to(device)

# Print model summary using torchinfo
summary(model, input_size=(1, 3,32, 32), device=device)


Layer (type:depth-idx)                   Output Shape              Param #
segUnet                                  [1, 19, 32, 32]           --
├─ModuleList: 1-1                        --                        --
│    └─encoder_block: 2-1                [1, 8, 32, 32]            --
│    │    └─conv_block: 3-1              [1, 8, 32, 32]            840
│    │    └─MaxPool2d: 3-2               [1, 8, 16, 16]            --
│    └─encoder_block: 2-2                [1, 16, 16, 16]           --
│    │    └─conv_block: 3-3              [1, 16, 16, 16]           3,552
│    │    └─MaxPool2d: 3-4               [1, 16, 8, 8]             --
│    └─encoder_block: 2-3                [1, 32, 8, 8]             --
│    │    └─conv_block: 3-5              [1, 32, 8, 8]             14,016
│    │    └─MaxPool2d: 3-6               [1, 32, 4, 4]             --
├─conv_block: 1-2                        [1, 64, 4, 4]             --
│    └─Conv2d: 2-4                       [1, 64, 4, 4]             18,496
│  

In [10]:
# Function to recursively print the model's architecture
def print_model_structure(model, indent=0):
    print(len(model.encoders))
    x=model.encoders[0]

# Print the model architecture
print_model_structure(model)

3


In [2]:
import os

def delete_png_files(directory):
    # Loop through all files in the directory
    for filename in os.listdir(directory):
        # Check if the file ends with .png
        if filename.endswith(".png"):
            # Construct the full file path
            file_path = os.path.join(directory, filename)
            # Delete the file
            os.remove(file_path)
            print(f"Deleted {filename}")

# Example usage
delete_png_files('/home/gaston/Downloads/data/labels')


Deleted 9fdba515-e3b3f520.png
Deleted a4ad61e9-4d218ba5.png
Deleted 0d538703-ccd64878.png
Deleted 5b6e87d3-d1fe2836.png
Deleted 0c07e51c-fb369fc9.png
Deleted 1e77570b-cc5bcde4.png
Deleted 6c7e1865-12b9e8f7.png
Deleted 65f82959-60c2f9b4.png
Deleted 553e4a19-494d0000.png
Deleted 4411f74d-00000000.png
Deleted 52457b6d-451c75be.png
Deleted 7e5ef657-a9ad0000.png
Deleted adb9b134-2937d690.png
Deleted 9a888ffa-c3840000.png
Deleted a91b7555-00000825.png
Deleted abac9e9b-f4ed3629.png
Deleted 00bc0319-94afabc2.png
Deleted 5dbcc354-592a1a0d.png
Deleted 3198a12f-aff85b07.png
Deleted 8554d481-2385dc5a.png
Deleted a91b7555-00001255.png
Deleted fdb7ff6e-bcaa5d96.png
Deleted 57c53979-37f1da14.png
Deleted 5eda07c0-d3c25831.png
Deleted 0f442be6-6b658bbe.png
Deleted 4e4bbf1d-69160cc1.png
Deleted 1cbc711c-11b5cdf4.png
Deleted 7861b45f-00000000.png
Deleted 09b978e2-d23f59a8.png
Deleted 7e25ca64-638e903c.png
Deleted 6ea4cd53-edee13a9.png
Deleted 2ad8324d-615a5cd5.png
Deleted 87154a3d-65d8e0c4.png
Deleted 74